In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
rows = []

for page_num in range(1, 15):
    url = f"https://www.goodreads.com/list/show/6934.Science_Fiction_Books_by_Female_Authors?page={page_num}"
    # print("Now scraping", url)
    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    
    books = doc.find_all('tr')

    for book in books:
        row = {}
        row['Rank'] = book.find(class_="number").text
        row['Title'] = book.find(itemprop="name").text
        row['Author'] = book.find(class_="authorName").text
        row['Score'] = book.find(class_="smallText uitext").find('a').text
        row['Votes'] = book.find(class_="smallText uitext").find('img').find_previous_sibling('a').text
        row['Rating'] = book.find(class_="minirating").text
        rows.append(row)

df = pd.DataFrame(rows)

In [3]:
df['Number in series'] = df['Title'].str.extract(".+\(.*, #(.*)\)", expand=False)

In [4]:
df['Series'] = df['Title'].str.extract(".+\((.+).*#\d+\)", expand=False)
df['Series'] = df.Series.str.replace(',', '')

In [5]:
df['Title1'] = df['Title'].str.extract("(.+)\(.*", expand=False)
df.loc[df.Title1.isnull(), 'Title1'] = df['Title']
df['Title'] = df['Title1']

In [6]:
df['Score'] = df['Score'].str.extract("score: (.*)", expand=False)
df['Score'] = df['Score'].str.replace(',', '')

In [7]:
df['Avg rating'] = df['Rating'].str.extract("(.*) avg.+", expand=False)
df['Number of ratings'] = df['Rating'].str.extract(".* (.+) ratings", expand=False)
df['Number of ratings'] = df['Number of ratings'].str.replace(',', '')

In [8]:
df['Votes'] = df['Votes'].str.extract("(.*) p.+", expand=False)

In [9]:
df = df[['Rank', 'Title', 'Series', 'Number in series', 'Author', 'Score', 'Votes', 'Avg rating', 'Number of ratings']]

In [10]:
df.head()

,Rank,Title,Series,Number in series,Author,Score,Votes,Avg rating,Number of ratings
0,1,The Handmaid's Tale,NaN,NaN,Margaret Atwood,30733,314,4.09,1103055
1,2,The Hunger Games,The Hunger Games,1,Suzanne Collins,28553,292,4.33,5742381
2,3,Frankenstein,NaN,NaN,Mary Wollstonecraft Shelley,21909,224,3.78,1023619
3,4,A Wrinkle in Time,Time Quintet,1,Madeleine L'Engle,18720,196,4.01,903708
4,5,The Left Hand of Darkness,Hainish Cycle,NaN,Ursula K. Le Guin,17920,184,4.06,98822


In [11]:
df.to_csv("Science-Fiction.csv", index=False)